In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(1)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [ ]:
DATA_PATH = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
TRAIN_SOURCE_PATH = DATA_PATH/'train'
proj_id = 'bw2color'
TENSORBOARD_PATH = Path('data/tensorboard/WganModel')


dpath = DATA_PATH/(proj_id + '_2622_D.h5')
gpath = DATA_PATH/(proj_id + '_2622_G.h5')
keep_pct=1.0
bs=2
sz=224

c_lr=3e-4
c_lrs = np.array([c_lr/1000,c_lr/100,c_lr])

g_lr=1e-4
g_lrs = np.array([g_lr/1000,g_lr/100,g_lr])

torch.backends.cudnn.benchmark=True

## Training

In [ ]:
netG = Unet34(nf_factor=4).cuda()
netGVis = ModelStatsVisualizer(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = FeatureCritic(sz=sz,filter_scale=4).cuda()
netDVis = ModelStatsVisualizer(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [ ]:
md = get_matched_image_model_data(image_size=sz, batch_size=bs, root_data_path=DATA_PATH, train_root_path=TRAIN_SOURCE_PATH, proj_id=proj_id, keep_pct=keep_pct)

In [ ]:
trainer = WGANTrainer(netD=netD, netG=netG, md=md, bs=bs, sz=sz, dpath=dpath, gpath=gpath, citers=1)
trainerTB = WganTrainerStatsVisualizer(TENSORBOARD_PATH, trainer, 'trainer')

In [ ]:
#netG.freeze_to(1)
#netD.freeze_to(1)
trainer.train(lrs_critic=c_lrs, lrs_gen=g_lrs, clr_critic=(1,8, 0.9, 0.9), clr_gen=(1,8, 0.9, 0.9), cycle_len=1, epochs=1)